In [ ]:
import os
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
df_id = pd.read_csv(os.path.join(os.getcwd(), r"ADNI1-Total.csv"))

# df_id_not_type = df_id[~df_id["Imaging Protocol"].str.contains("Type")]
# df_id_not_type["Imaging Protocol"] = (
#    df_id_not_type["Imaging Protocol"] + ";Acquisition Type=3D"
# )
# df_id_type = df_id[df_id["Imaging Protocol"].str.contains("Type")]
# df_id = pd.concat([df_id_not_type, df_id_type])


# df_id[["Manufacturer", "Model", "Type"]] = df_id["Imaging Protocol"].str.extract(
#    "Manufacturer=(.+);Mfg Model=(.+);Acquisition Type=(.+)", expand=True
# )
df_id[["Manufacturer", "Model", "Type"]] = df_id["Imaging Protocol"].str.extract(
    "Manufacturer=(.+);Mfg Model=(.+);Acquisition Type=(.+)", expand=True
)
# df_id["Type"] = df_id["Type"].fillna("3D")
df_id = df_id[df_id["Type"] == "3D"]

df_id = df_id.replace(
    {
        "GE MEDICAL SYSTEMS": "GE",
        "Philips Healthcare": "Philips",
        "Philips Medical Systems": "Philips",
        "SIEMENS|PixelMed": "Siemens",
        "SIEMENS": "Siemens",
    },
    regex=False,
)
print(df_id.Description.nunique())
df_id = df_id[
    (df_id["Description"].str.contains("MP"))
    & (~df_id["Description"].str.contains("TEMPLE"))
    & (~df_id["Visit"].str.contains("ADNI Baseline"))
    & (~df_id["Visit"].str.contains("No Visit Defined"))
]

df_ge = df_id[
    (df_id["Manufacturer"] == "GE")
    & (df_id["Model"].isin(["SIGNA EXCITE", "SIGNA HDx", "GENESIS_SIGNA"]))
]
df_philips = df_id[
    (df_id["Manufacturer"] == "Philips")
    & (df_id["Model"].isin(["Intera", "Intera Achieva", "Achieva", "Gyroscan Intera"]))
]
df_siemens = df_id[
    (df_id["Manufacturer"] == "Siemens")
    & (
        df_id["Model"].isin(
            ["Symphony", "Sonata", "Allegra", "Avanto", "Trio", "TrioTim"]
        )
    )
]
df_id = pd.concat([df_ge, df_philips, df_siemens])

In [ ]:
df_id["Image ID"].nunique()

In [ ]:
df_id["Imaging Protocol"].iloc[0]

In [ ]:
teste = df_id[(df_id["Manufacturer"] == "Philips")]
teste.Model.unique()

In [ ]:
df_id.groupby("Manufacturer")["Image ID"].nunique()

In [ ]:
df_id["Subject ID"].nunique()

In [ ]:
df_id["Image ID"].nunique()

In [ ]:
df_id[(df_id["Subject ID"] == "010_S_0419")].sort_values(by=["Visit"])

In [ ]:
unique_groups = df_id.groupby("Subject ID")["Research Group"].unique()

inconsistent_subjects = unique_groups[unique_groups.apply(len) > 1]
print("Sujeitos com mais de um Research Group:")
print(inconsistent_subjects)

In [ ]:
df_id.groupby(["Visit"])["Subject ID"].nunique()

In [ ]:
df_id.groupby(["Visit"])["Image ID"].nunique()

In [ ]:
df_id.groupby(["Visit", "Manufacturer", "Research Group"])["Image ID"].nunique()

In [ ]:
df_id.groupby(["Visit", "Manufacturer", "Research Group", "Sex"])["Image ID"].nunique()

In [ ]:
df_id.groupby(["Visit", "Research Group"])["Image ID"].nunique()

In [ ]:
df_id.groupby(["Visit", "Research Group"])["Image ID"].nunique()

In [ ]:
df_id.groupby(["Manufacturer", "Research Group"])["Image ID"].nunique()

In [ ]:
# test = df_id[df_id["Visit"].isin(["ADNI Screening"])]
df_id.groupby(["Research Group"])["Image ID"].nunique()

In [ ]:
df_id["Image ID"].nunique()

In [ ]:
df_id[df_id["Subject ID"] == "002_S_0413"]

In [ ]:
df_id[df_id["Subject ID"] == "010_S_0419"]

In [ ]:
df_id[df_id["Subject ID"] == "128_S_0167"]

In [ ]:
df_id.sort_values(["Subject ID", "Visit", "Manufacturer"], inplace=True)
firsts = df_id.groupby(["Subject ID", "Visit"], as_index=False).first()


def one_manufacturer(df):
    output = pd.DataFrame()
    all_sub_id = df["Subject ID"].unique()
    for sub_id in all_sub_id:
        df_sub_id = df[df["Subject ID"] == sub_id]
        manufacturer_unique = df_sub_id["Manufacturer"].unique()[0]
        df_sub_id = df_sub_id[(df_sub_id["Manufacturer"] == manufacturer_unique)]
        output = pd.concat([output, df_sub_id])
    return output


firsts = one_manufacturer(firsts)

In [ ]:
firsts[firsts["Subject ID"] == "010_S_0419"]

In [ ]:
firsts[firsts["Subject ID"] == "128_S_0167"]

In [ ]:
# test = df_id[df_id["Visit"].isin(["ADNI Screening"])]
firsts["Image ID"].nunique()

In [ ]:
# test = df_id[df_id["Visit"].isin(["ADNI Screening"])]
firsts.groupby(["Research Group"])["Image ID"].nunique()

In [ ]:
firsts["Image ID"].nunique()

In [ ]:
firsts.groupby(["Visit", "Research Group"])["Image ID"].nunique()

In [ ]:
firsts.groupby(["Manufacturer", "Research Group", "Sex"])["Image ID"].nunique()

In [ ]:
firsts.groupby(["Manufacturer"])["Image ID"].nunique()

In [ ]:
# GE 360
# SIEMENS 360
# PHILIPS 285

# firsts.groupby(["Manufacturer", 'RE'])["Image ID"].nunique()

In [ ]:
firsts["Image ID"].nunique()

In [ ]:
(firsts.groupby(["Subject ID"])["Manufacturer"].nunique() > 1).sum()

In [ ]:
(firsts.groupby(["Subject ID"])["Research Group"].nunique() > 1).sum()

In [ ]:
firsts["dot"] = 1
firsts[["Image ID", "dot"]].to_csv("ID.csv", index=False)